In [1]:
print("Hello")

Hello


## Imports

In [2]:
from helpers.common import vector_store

/home/yash/Desktop/Code/KGP/rag_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name togethercomputer/m2-bert-80M-8k-retrieval. Creating a new one with mean pooling.
You are using a model of type m2_bert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
/home/yash/.cache/huggingface/modules/transformers_modules/togethercomputer/m2_hyphen_bert_hyphen_80M_hyphen_8k_hyphen_retrieval/90e0b28f3382c289cc6c1e92ef53d7dc5a3ec14b/blockdiag_multiply.py:50: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.bfloat16)
/home/yash/.cache/huggingface/modules/transformers_module

-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


In [3]:
from helpers.common import COLLECTION_NAME_TOGETHER, llm
from langchain.messages import HumanMessage, SystemMessage
from helpers.schema import ChunkMetadata
from qdrant_client.models import Filter, FieldCondition, MatchValue




In [4]:
vector_store.client.get_collection(COLLECTION_NAME_TOGETHER)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, warnings=None, indexed_vectors_count=2692, points_count=2703, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors={'': VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'langchain-sparse': SparseVectorParams(index=None, modifier=None)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimi

## Extract Metadata

In [8]:

def extract_filters(user_query: str):

    system_message = """
You extract metadata filters from financial queries.

Return None for fields not mentioned.

COMPANY MAPPINGS:
- Amazon/AMZN -> amazon
- Google/Alphabet/GOOGL/GOOG -> google
- Apple/AAPL -> apple
- Microsoft/MSFT -> microsoft
- Tesla/TSLA -> tesla
- Nvidia/NVDA -> nvidia
- Meta/Facebook/FB -> meta

DOC TYPE:
- Annual report -> 10-k
- Quarterly report -> 10-q
- Current report -> 8-k

Examples:
"Amazon Q3 2024 revenue" -> {"company_name": "amazon", "doc_type": "10-q", "fiscal_year": "2024", "fiscal_quarter": "q3"}
"Apple 2023 annual report" -> {"company_name": "apple", "doc_type": "10-k", "fiscal_year": "2023"}
"Tesla profitability" -> {"company_name": "tesla"}

Extract metadata strictly based on the user query.
"""

    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=user_query),
    ]

    structured_llm = llm.with_structured_output(ChunkMetadata)

    response = structured_llm.invoke(messages)

    if hasattr(response, "parsed"):
        metadata = response.parsed
    else:
        metadata = response

    return metadata.model_dump(exclude_none=True)


In [22]:
query = "what is amazon's revenue in 2023 in q1?"
filters = extract_filters(query)

/home/yash/Desktop/Code/KGP/rag_venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=ChunkMetadata(company_nam...3', fiscal_quarter='q1'), input_type=ChunkMetadata])
  return self.__pydantic_serializer__.to_python(


In [23]:
filters

{'company_name': 'amazon',
 'doc_type': '10-q',
 'fiscal_year': '2023',
 'fiscal_quarter': 'q1'}

In [6]:
[FieldCondition(key=f"metadata.{key}", match=MatchValue(value=value))  for key, value in filters.items()]

NameError: name 'filters' is not defined

In [7]:
def hybrid_search(query: str, k: int = 5):
    """
    Perform hybrid search (dense + sparse vectors).
    
    Args:
        query: Search query
        k: Number of results
        filters: Optional filters like {"company_name": "amazon", "fiscal_year": 2024}
    
    Returns:
        List of Document objects
    """

    filters = extract_filters(query)

    qdrant_filter = None

    if filters:
        condition = [FieldCondition(key=f"metadata.{key}", match=MatchValue(value=value))
                     for key, value in filters.items()]
        
        qdrant_filter = Filter(must=condition)

    results = vector_store.similarity_search(query=query, k=k, filter=qdrant_filter)

    return results


In [ ]:
query = "what is the tesla's revenue"
results = hybrid_search(query, k=10)

In [32]:
results

[]

In [9]:
query = "what is amazon's cashflow in 2024 in q1?"
results = hybrid_search(query, k=10)

/home/yash/Desktop/Code/KGP/rag_venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=ChunkMetadata(company_nam...4', fiscal_quarter='q1'), input_type=ChunkMetadata])
  return self.__pydantic_serializer__.to_python(


In [10]:
results

[Document(metadata={'company_name': 'amazon', 'doc_type': '10-q', 'fiscal_quarter': 'q1', 'fiscal_year': '2024', 'content_type': 'text', 'file_hash': '14da6c72f029ee2191b9fc7345eab770ed70080549f4911888dc990a11f41a62', 'source_file': 'amazon 10-q q1 2024.md', 'page': 47, '_id': 'f2381003-b74d-4225-bc11-9810fac15a90', '_collection_name': 'financial_docs_together'}, page_content="\n\n|   Item 6. | Exhibits                                                                                                                                                                                                                                                                                                                                                                                                                                      |\n|-----------|------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
collection_info = vector_store.client.get_collection(COLLECTION_NAME_TOGETHER)

In [25]:
collection_info.model_dump()

{'status': <CollectionStatus.GREEN: 'green'>,
 'optimizer_status': <OptimizersStatusOneOf.OK: 'ok'>,
 'warnings': None,
 'indexed_vectors_count': 2692,
 'points_count': 2703,
 'segments_count': 8,
 'config': {'params': {'vectors': {'': {'size': 768,
     'distance': <Distance.COSINE: 'Cosine'>,
     'hnsw_config': None,
     'quantization_config': None,
     'on_disk': None,
     'datatype': None,
     'multivector_config': None}},
   'shard_number': 1,
   'sharding_method': None,
   'replication_factor': 1,
   'write_consistency_factor': 1,
   'read_fan_out_factor': None,
   'on_disk_payload': True,
   'sparse_vectors': {'langchain-sparse': {'index': None, 'modifier': None}}},
  'hnsw_config': {'m': 16,
   'ef_construct': 100,
   'full_scan_threshold': 10000,
   'max_indexing_threads': 0,
   'on_disk': False,
   'payload_m': None,
   'inline_storage': None},
  'optimizer_config': {'deleted_threshold': 0.2,
   'vacuum_min_vector_number': 1000,
   'default_segment_number': 0,
   'max_se

In [28]:
collection_info.config.params.vectors

{'': VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}